In [10]:
from tools.ground_removal import *
from tools.raycasting import *
import numpy as np
from sklearn.cluster import DBSCAN
from mayavi import mlab
from tools.visualization import *
from scipy.spatial import KDTree
import yaml
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
first_frame = 0
second_frame = 2
pts, intens = get_synchronized_frames_without_ground(first_frame,1)
pts2, intens2 = get_synchronized_frames_without_ground(second_frame,1)

In [28]:
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], color=(0,1,0), mode='point')
mlab.points3d(pts2[:,0], pts2[:,1], pts2[:,2], color=(0,0,1), mode='point')
mlab.show()

In [25]:
with open('raycast_config.yaml', 'r') as f:
    config = yaml.safe_load(f)
figure = mlab.figure()
mask = find_dynamic_objects(pts, pts2, get_synchronized_origin(second_frame), config['CAR'], figure)

Number of checks for raycast is 139
Number of checks for raycast is 105
ray has reached the destination without hitting anything
Number of checks for raycast is 168
dynamic objects: [48]


In [26]:
mlab.points3d(pts[:,0][mask], pts[:,1][mask], pts[:,2][mask], color=(1,0,0), mode='point')
mlab.points3d(pts[:,0][~mask], pts[:,1][~mask], pts[:,2][~mask], color=(1,1,1), mode='point')
mlab.show()